In [1]:
import numpy as np
import pandas as pd
from numpy.linalg import inv, matrix_rank
from numpy import matmul
from scipy.stats import f

In [2]:
# R dXK, d linear restrictions and K predictors. Rank(R) = d

In [3]:
alpha = 0.05
df1 = 1 # rank of R. df1
df2 = 192 # n-K
f.ppf(alpha, df1, df2) # critical value

0.003942433704199765

In [39]:
def sim_data(betas, rows = 1000):
    xs = np.random.random((rows, len(betas)))
    y = (xs @ betas).reshape((rows,1)) +np.random.normal(0,1,(rows,1))
    return xs, y
    
    
def estimate_beta(xs, y):
    est = inv(xs.T @ xs) @xs.T @ y
    return est
    

def f_stat(R, betas, xs, r, y):
    d = matrix_rank(R)
    #print(d)
    n = len(xs)
    K = len(betas)
    e = y - xs @ betas
    #print(e.shape)
    upper = ((R @ betas - r).T @ inv(R @ inv(xs.T @ xs) @ R.T) @ (R@betas -r))/d
    #print(upper)
    sample_var = (e.T @ e)/(n-K)
    #print(sample_var)
    F = upper/sample_var
    return [F, d, n-K]


def make_R_r(*args, betas):
    R = np.zeros((len(args), len(betas)))
    for idx,arg in enumerate(args):
        R[idx, arg] = 1
    
    r = (R @ betas).reshape((len(args),1))
    return R, r
        

def test(F,df1, df2, alpha = 0.05):
    crit = f.ppf(1-alpha, df1, df2)
    if F <= crit:
        print('Accept H0')
    else:
        print('Reject H0')
    return crit



def sim(betas, R, r, res = True): # puts much of the functions together. *args are desired restrictions
    xs,y = sim_data(betas)
    beta_est = estimate_beta(xs,y)
    F, df1, df2 = f_stat(R, beta_est, xs, r, y)
    if res == True:
        crit = test(F, df1, df2)
        print(f'{'-'*60}')
        print(f'Critical value: {round(crit, 3)}')
        print(f'F: {round(F[0][0],3)}')
        print(f'beta estimates: {[n for n in beta_est]}')
        print(f'True betas: {[n for n in betas]}')
    else:
        crit = f.ppf(1-0.05, df1, df2)
    return crit, F
    

In [40]:
betas = np.array([1,5,3,8]).reshape((4,1))
R, r = make_R_r(1,3, [1,2,3], betas=betas)
crit, F = sim(betas, R, r)

Accept H0
------------------------------------------------------------
Critical value: 2.614
F: 0.474
beta estimates: [array([0.95745554]), array([5.10621081]), array([2.94763745]), array([7.94125097])]
True betas: [array([1]), array([5]), array([3]), array([8])]


In [41]:
def loop(betas,R, r, iter = 1000):
    n = 0
    reject = 0
    for i in range(iter):
        crit, F = sim(betas, R, r, res=False)
        if F>crit:
            reject += 1
        else:
            pass
        n += 1
    return reject, n
        
    

## 3) b)

In [42]:
a = np.array([1,5,3,8]).reshape((4,1))
R_a, r_a = make_R_r(1,3, [1,2,3], betas=a)

b = np.array([2,4,1,9]).reshape((4,1))
R_b, r_b = make_R_r(1,2, [1,3], betas=b)

c = np.array([1,2,1,2]).reshape((4,1))
R_c, r_c = make_R_r([1,2], 0, betas=c)

d = np.array([1,5,2,8,4]).reshape((5,1))
R_d, r_d = make_R_r([1,2], 0, betas=d)

test_sets = [(a, R_a, r_a), (b, R_b, r_b), (c, R_c, r_c), (d, R_d, r_d)]
i = 1
for betas, R, r in test_sets:
    rej, n = loop(betas, R,r)
    print(f'Test {i}')
    print(f'True beta values {list(betas)}')
    print(f'Rejected tests: {rej}')
    print(f'Out of: {n}')
    print(f'{round(rej/n *100,2)}% of tests were rejected at level 0.05')
    print(f'{'-'*60}')
    
    i += 1



Test 1
True beta values [array([1]), array([5]), array([3]), array([8])]
Rejected tests: 58
Out of: 1000
5.8% of tests were rejected at level 0.05
------------------------------------------------------------
Test 2
True beta values [array([2]), array([4]), array([1]), array([9])]
Rejected tests: 37
Out of: 1000
3.7% of tests were rejected at level 0.05
------------------------------------------------------------
Test 3
True beta values [array([1]), array([2]), array([1]), array([2])]
Rejected tests: 57
Out of: 1000
5.7% of tests were rejected at level 0.05
------------------------------------------------------------
Test 4
True beta values [array([1]), array([5]), array([2]), array([8]), array([4])]
Rejected tests: 55
Out of: 1000
5.5% of tests were rejected at level 0.05
------------------------------------------------------------
